In [1]:
import numpy as np

#Bayesian classifier:
class BernoulliNaiveBayes:
  def __init__(self, dataset, labels, alpha = [1.0, 1.0], beta = [1.0, 1.0]):
    """
    labels : 1d array, incude index of classes, start at 0
    dataset: (number_sample, num_features)
    prior function: Beta distribution
    num_classes = 2
    likelihood function: Bernoulli distribution
    """
    self.N = dataset.shape[0]
    self.num_classes = 2
    self.num_features = dataset.shape[1]
    self.alpha = alpha 
    self.beta = beta
    self.lamda = self.get_lamda(dataset, labels)
    self.prior_value = self.get_prior_value(labels)
    
  def get_lamda(self, dataset, labels):
    """
    Apply laplace smoothing.
    return all lambda parmeter for every feature variable
    """
    lp = 0
    if ((self.alpha[0] - 1).all() or (self.beta[0] - 1).all()) or ((self.alpha[1] - 1).all() or (self.beta[1] - 1).all()):
      #Apply laplace smoothing:
      lp = 1
    result = []
    for c_index in np.arange(self.num_classes):
      result.append((np.sum(((labels == c_index)[:, None]) & dataset, axis=0) + self.alpha[c_index] - 1 + lp) / \
            (np.sum(labels == c_index, dtype=np.float32) + self.alpha[c_index] + self.beta[c_index] - 2 + 2*lp))
    return np.array(result)

  def get_prior_value(self, labels):
    """
    return all prior probability of all classes
    """
    result = np.empty((self.num_classes, ))
    for c_index in np.arange(self.num_classes):
      result[c_index] = np.sum(labels == c_index, dtype=np.float32) / self.N; 
    return result

  def predict(self, x):
    """
    Predict a class for new sample
    x = (num_sample, num_features) 
    """  
    posterior_value = []
    for c_index in np.arange(self.num_classes):
      posterior_value.append(
          np.log(self.prior_value[c_index]) + \
          np.sum(
            (x) * np.log(self.lamda[c_index]) + \
             (-x + 1) * np.log(1 - self.lamda[c_index])
            ,axis=1)
      )
    posterior_value = np.array(posterior_value)
    return np.argmax(posterior_value, axis=0)
  
  def predict_proba(self, x):
    posterior_value = np.empty((x.shape[0], self.num_classes))
    for c_index in np.arange(self.num_classes):
      posterior_value[:, c_index] = \
              self.prior_value[c_index] *  \
              np.prod(
                  (self.lamda[c_index]**(x)) * ((1 - self.lamda[c_index])**(1-x)),
                  axis=1
              )
    return posterior_value / np.sum(posterior_value, axis=1)[:, None]
  
  def evaluate(self, x, y):
    """
    return:
      miss_classify:  sum(predicted_class == y)
    error_function = Sum((1 - P(C|X) * P(X))
    """
    predict_index_classes = self.predict(x)
    return np.sum(np.equal(predict_index_classes, y), )                        
   
  def show(self):
    print(self.lamda)